# Example script to generate the Bowl_shaped SVG file

In [2]:
debug = False

In [ ]:
import cairo
from sympy import Point, Segment
import sympy as sym
import numpy as np
from IPython.display import SVG

# Function to draw text at a given position
def draw_text(position, text, size=12):
    context.save()
    context.scale(1, -1)  # Flip text back to normal orientation
    context.set_font_size(size)
    context.move_to(position[0], -position[1])
    context.show_text(text)
    context.restore()

# Function to draw a dotted quarter-circle, useful for marking boundaries
def draw_dotted_quarter_circle(center, radius, start_angle=0, end_angle=90, rgba=(0, 0, 0, 0.5)):
    context.set_source_rgba(*rgba)
    context.set_dash([2, 2])  # Create a dotted effect
    context.arc(center[0], center[1], radius, np.radians(start_angle), np.radians(end_angle))
    context.stroke()
    context.set_dash([])  # Reset to solid lines

# Function to draw a solid line between two points
def draw_line(point1, point2, width=2):
    context.set_line_width(width)
    context.move_to(point1[0], point1[1])
    context.line_to(point2[0], point2[1])
    context.stroke()

# Function to draw a line using sympy Point objects
def draw_line_s(point_1, point_2, width=3):
    point1 = point_1.coordinates
    point2 = point_2.coordinates
    context.set_line_width(width)
    context.move_to(point1[0], point1[1])
    context.line_to(point2[0], point2[1])
    context.stroke()

# Function to draw a segment using sympy's Segment class
def draw_segment(segment, width=1, rgba=(0, 0, 0, 1)):
    context.set_source_rgba(rgba[0], rgba[1], rgba[2], rgba[3])
    context.set_line_width(width)
    point1 = segment.points[0].coordinates
    point2 = segment.points[1].coordinates
    context.move_to(point1[0], point1[1])
    context.line_to(point2[0], point2[1])
    context.stroke()

# Function to draw a connected path through a list of Points
def draw_path(List, width=1, rgba=(0, 0, 0, 1)):
    context.set_source_rgba(rgba[0], rgba[1], rgba[2], rgba[3])
    context.set_line_width(width)
    for index, obj in enumerate(List):
        if isinstance(obj, Point):
            if index == 0:
                context.move_to(obj[0], obj[1])
            else:
                context.line_to(obj[0], obj[1])
    context.stroke()

# Function to calculate intersection of two line segments
def line_intersection(segment1, segment2):
    A = segment1.points[0].coordinates
    B = segment1.points[1].coordinates
    C = segment2.points[0].coordinates
    D = segment2.points[1].coordinates

    xdiff = (A[0] - B[0], C[0] - D[0])
    ydiff = (A[1] - B[1], C[1] - D[1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        raise Exception('Lines do not intersect')

    d = (det(A, B), det(C, D))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return Point(x, y)

# Function to draw axes with labels
# Function to draw axes with labels
# Function to draw axes with labels dynamically based on axis length
def draw_axes(x_limit=400, y_limit=200, tick_spacing=50):
    context.set_source_rgba(0, 0, 0, 1)
    context.set_line_width(1)

    # Draw X-axis
    context.move_to(-x_limit, 0)
    context.line_to(x_limit, 0)
    context.stroke()
    draw_text((x_limit + 10, -5), "X-axis", 12)  # Label for X-axis

    # Draw Y-axis
    context.move_to(0, -y_limit)
    context.line_to(0, y_limit)
    context.stroke()
    draw_text((-15, y_limit + 10), "Y-axis", 12)  # Label for Y-axis

    # Add tick marks and labels on X-axis
    for i in range(-x_limit, x_limit + 1, tick_spacing):
        context.move_to(i, -5)
        context.line_to(i, 5)
        context.stroke()
        draw_text((i - 10, -15), str(i), 8)  # Adjust label position

    # Add tick marks and labels on Y-axis
    for i in range(-y_limit, y_limit + 1, tick_spacing):
        context.move_to(-5, i)
        context.line_to(5, i)
        context.stroke()
        draw_text((-30, i - 5), str(i), 8)  # Adjust label position



# Define output file for SVG
filename = "example_shape_1280.svg"

# Create an SVG surface with a specified size
with cairo.SVGSurface(filename, 800 / 0.352778, 400 / 0.352778) as surface:
    context = cairo.Context(surface)
    context.translate(50 / 0.352778, 350 / 0.352778)
    context.scale(1 / 0.352778, -1 / 0.352778)  # Flip Y-axis

    if debug:
        draw_axes(x_limit=700, y_limit=300, tick_spacing=50)

    # Define coordinate system
    o = (0, 0)  # Origin
    y = (0, 350)  # Y-axis size
    x = (750, 0)  # X-axis size
    draw_line(o, x, 1)
    draw_line(o, y, 1)

    # Define projector parameters
    Beamers_position = Point(0, 0)
    Beamers_throw_ratio = 1.2
    Beamers_resolution_width = 1280
    Distance_beamer_fly = 200
    Arena_start_angle = 15  
    Arena_stop_angle = 140
    Arena_max_radius = 300
    Pixel_per_degree = 5
    Flys_position = Point(Distance_beamer_fly, 0)

    # Calculate number of rays from projector
    List_Beamer_rays = []
    List_Fly_rays = []
    n = int(Beamers_resolution_width / 2 / Pixel_per_degree)
    print("Total amount of rays =", n)

    # Generate beamer projection rays
    Beamer_projection_height = (Distance_beamer_fly + Arena_max_radius) / Beamers_throw_ratio / 2
    Beamer_endpos_rays = np.linspace(0, Beamer_projection_height, n)
    for point in Beamer_endpos_rays:
        List_Beamer_rays.append(Segment(Beamers_position, Point(Distance_beamer_fly + Arena_max_radius, point)))

    # Draw beamer rays in blue
    for obj in List_Beamer_rays:
        draw_segment(obj, 0.2, (0, 0, 1, 1))

    # Generate fly projection rays
    r = Arena_max_radius
    Start_index = int(n / Arena_stop_angle * Arena_start_angle)
    Stop_index = int(n / Arena_stop_angle * Arena_stop_angle)

    angles = np.linspace(0, Arena_stop_angle, n)
    for angle in angles:
        Fly_endpos_rays = Point(Distance_beamer_fly + r * sym.cos(angle * sym.pi / 180),
                                r * sym.sin(angle * sym.pi / 180))
        List_Fly_rays.append(Segment(Flys_position, Fly_endpos_rays))

    # Draw fly rays in red
    for obj in List_Fly_rays:
        draw_segment(obj, 0.2, (1, 0, 0, 1))

    # Calculate intersection points between rays
    intersectionpoints = []
    for index, obj in enumerate(List_Beamer_rays[Start_index:Stop_index], start=Start_index):
        intersectionpoints.append(line_intersection(List_Beamer_rays[index], List_Fly_rays[index]))

    print("Calculated Arena intersection points:", Stop_index - Start_index)
    draw_path(intersectionpoints)

    # Debug visualizations
    if debug:
        draw_dotted_quarter_circle((Distance_beamer_fly, 0), Arena_max_radius, 0, 90, (0, 0, 0, 0.3))
        draw_text((-20, -20), "Projector Position", 5)
        draw_text((Distance_beamer_fly + 5, -15), "Fly Position", 5)
        draw_text((Distance_beamer_fly + Arena_max_radius / 2, 50), f"Max Arena Radius: {Arena_max_radius} mm", 5)

# Display the SVG
SVG(filename=filename)


Total amount of rays = 6


Exception: Lines do not intersect